# Subqueries

> Udviklet af Thomas Lange & Mick Ahlmann Brun

Mere info: [https://github.com/M1ckB/T-SQL](https://github.com/M1ckB/T-SQL)

Version 1.0 2023-01-25

Laboratoriet kræver:

- En understøttet version af SQL Server
- En Stack Overflow database: [Brent Ozar](https://www.BrentOzar.com/go/querystack) (medium)

Læs mere om subqueries i Microsofts T-SQL reference:

- [https://learn.microsoft.com/en-us/sql/relational-databases/performance/subqueries?view=sql-server-ver16](https://learn.microsoft.com/en-us/sql/relational-databases/performance/subqueries?view=sql-server-ver16)


## Indholdsfortegnelse

- [Introduktion til subqueries](#Introduktion-til-subqueries)
- [Uafhængige subqueries](#Uafhængige-subqueries)
  - [Skalar subquery](#Skalar-subquery)
  - [Flerværdi subquery](#Flerværdi-subquery)
- [Korrelerede subqueries](#Korrelerede-subqueries)
  - [EXISTS-prædikatet](#EXISTS-prædikatet)
- [Hovedpointer](#Hovedpointer)


## Introduktion til subqueries

SQL understøtter at man kan indlejre forespørgsler i hinanden.

Man siger at den forespørgsel hvis resultat returneres til forespørgeren er den *ydre* forespørgsel, mens den *indre* forespørgsel er en der bruges af den ydre forespørgsel. Sidstnævnte kaldes en *subquery*.

Man kan tænke på at subqueries står i stedet for et udtryk baseret på konstanter eller variable som evalueres på kørselstidspunktet. Subqueries er dynamiske i sin natur, dvs. resultatet af en subquery vil ændre sig i takt med at indholdet i de underliggende tabeller ændrer sig.

Subqueries kan have nogle forskellige egenskaber:

- De kan være både uafhængige eller korreleret med den ydre forespørgsel
- De kan returnere en enkelt værdi, en liste af værdier eller en hel tabel
 
 Mens subqueries der returnerer hele tabeller vil gennemgås som et særskilt emne, kaldt [derived tables](Derived-tables.ipynb), vil de øvrige typer af subqueries gennemgås i nærværende notebook.

Use cases:

- Dynamisk beregningstrin i forespørgsel
- Beregning af positiv- og negativlister til filtrering


In [ ]:
/* Nedenstående er et eksempel på en subquery som, på dynamisk vis, sørger for at det kun er
transaktioner fra det seneste batch som returneres */

CREATE TABLE #TableA (
  Id int NOT NULL,
  BatchNo int NOT NULL
);

INSERT INTO #TableA (Id, BatchNo)
VALUES
(1, 1), (2, 1), (3, 1),
(4, 2), (5, 2), (6, 3);

SELECT * FROM #TableA;

SELECT
    Id,
    BatchNo
FROM #TableA
WHERE BatchNo = (
    SELECT MAX(BatchNo)
    FROM #TableA
);

DROP TABLE #TableA;

## Uafhængige subqueries

Uafhængige subqueries er *subqueries* som er *uafhængige* af tabellerne fra den ydre forespørgsel som indeholder den. De evalueres logisk før den ydre forespørgsel, og den ydre forspørgsel bruger resultatet fra den indre forespørgsel.

Uafhængige subqueries kan returnere en [enkelt værdi](#Skalar-subquery) eller en [liste af værdier](#Flerværdi-subquery).


### Skalar subquery

En skalar subquery er en subquery som returnerer en enkelt værdi.

En skalar subquery må optræde over alt i den ydre forespørgsel hvor udtryk med enkeltværdier er tilladt, fx `SELECT` og `WHERE`.


In [ ]:
/* Nedenstående er eksempler på skalar subqueries */

CREATE TABLE #TableA (
  Id int NOT NULL,
  TableB_Id int NOT NULL
);

INSERT INTO #TableA (Id, TableB_Id)
VALUES
(1, 1), (2, 1), (3, 1),
(4, 2), (5, 2), (6, 3);

CREATE TABLE #TableB (
  Id int NOT NULL,
  Col nvarchar(10) NULL
);

INSERT INTO #TableB (Id, Col)
VALUES
(1, 'A'), (2, 'B'), (3, 'B');

SELECT * FROM #TableA;
SELECT * FROM #TableB;

/* Bemærk at uafhængige subqueries kan markeres og eksekveres */

/* Udskift WHERE-delsætningen med de andre, udkommenterede muligheder */

SELECT
    Id
FROM #TableA
WHERE TableB_Id = (
    SELECT Id
    FROM #TableB
    WHERE Col LIKE '%A%'
    --WHERE Col LIKE '%B%'
    --WHERE Col LIKE '%C%'
);

/* En skalar subquery skal returnere en enkelt værdi, ellers fejler den. Såfremt en subquery ikke
    returnerer nogle værdier, så konverteres resultatet til NULL og ingen rækker returneres */

DROP TABLE #TableA;
DROP TABLE #TableB;

### Flerværdi subquery

En flerværdi subquery er en subquery som returnerer flere værdier som en enkelt kolonne, eller en liste af værdier.

En flerværdi subquery bruges i nogle prædikater, fx `IN` (samt `SOME`, `ANY` og `ALL`, men disse bruges sjældent).

`IN`-prædikatet har formen:

```sql
SELECT
    *
FROM TableA
WHERE <skalar-udtryk> IN (<flerværdi subquery>);
```

Prædikatet evalueres til *SANDT* såfremt skalar-udtrykket er lig en af værdierne returneret af subquerien.


In [ ]:
/* Nedenstående er eksempler på flerværdi subqueries */

CREATE TABLE #TableA (
  Id int NOT NULL,
  TableB_Id int NULL
);

INSERT INTO #TableA (Id, TableB_Id)
VALUES
(1, 1), (2, 1), (3, 1),
(4, 2), (5, 2), (6, 3);

CREATE TABLE #TableB (
  Id int NULL,
  Col nvarchar(10) NULL
);

INSERT INTO #TableB (Id, Col)
VALUES
(1, 'A'), (2, 'B'), (3, 'B'),
(4, 'C');

SELECT * FROM #TableA;
SELECT * FROM #TableB;

/* Der er mange problemer som kan løses på flere måder, fx ved brug af enten joins eller
    subqueries. Det er ikke til at sige hvad der er bedst. Følg din intuition og tun efter
    behov */

SELECT
    Id,
    TableB_Id
FROM #TableA
WHERE TableB_Id IN (
    SELECT Id
    FROM #TableB
    WHERE Col LIKE '%B%'
);

SELECT
    a.Id,
    a.TableB_Id
FROM #TableA AS a
INNER JOIN #TableB AS b ON b.Id = a.TableB_Id
WHERE b.Col LIKE '%B%';

/* Bemærk at du ikke behøves at bekymre dig om at inkludere DISTINCT-delsætningen i din subquery
    da databasesystemet selv fjerner eventuelle dubletter */

SELECT
    Id,
    Col
FROM #TableB
WHERE Id IN (
    SELECT TableB_Id
    FROM #TableA
);

/* Vær opmærksom på NULL-værdier! */

SELECT
    Id,
    Col
FROM #TableB
WHERE Id NOT IN (
    SELECT TableB_Id
    FROM #TableA
);

INSERT INTO #TableA (Id, TableB_Id)
VALUES
(7, NULL);

SELECT
    Id,
    Col
FROM #TableB
WHERE Id NOT IN (
    SELECT TableB_Id
    FROM #TableA
);

/* Id=4 fra #TableB har ikke umiddelbart en matchende række i #TableA, men fordi der også optræder
    en NULL-værdi i #TableA, så kan vi ikke vide os sikker */

DROP TABLE #TableA;
DROP TABLE #TableB;

### *Tid til opgaver...*

Lav opgave 1, 2 og 3 i [opgavehæftet](Subqueries.sql).


## Korrelerede subqueries

Korrelerede subqueries er subqueries som refererer tabeller og kolonner fra den ydre forespørgsel. Korrelerede subqueries er således afhængige af den ydre forespørgsel og kan ikke eksekveres for sig. De evalueres logisk separat for hver række i den ydre forespørgsel.


In [ ]:
/* Nedenstående er eksempler på korrelerede subqueries */

CREATE TABLE #TableA (
  Id int NOT NULL,
  Cat nchar(1) NULL
);

INSERT INTO #TableA (Id, Cat)
VALUES
(1, 'A'), (2, 'A'), (3, 'A'),
(4, 'B'), (5, 'B'), (6, 'C');

SELECT * FROM #TableA;

/* Bemærk at korrelerede queries ikke umiddelbart kan markeres og eksekveres */

/* I eksemplet findes for hver kategori den seneste transaktion (med højeste Id) */

SELECT
    Id,
    Cat
FROM #TableA as a1
WHERE Id IN (
    SELECT MAX(Id)
    FROM #TableA AS a2
    WHERE a1.Cat = a2.Cat
);

DROP TABLE #TableA;

### `EXISTS`-prædikatet

T-SQL understøtter et prædikat kaldt `EXISTS` som tager en (korreleret) subquery som input og returnerer *SANDT* hvis subquerien returnerer en eller flere rækker (matches) og ellers *FALSKT*.

`EXISTS`-prædikatet har formen:

```sql
SELECT
    *
FROM TableA
WHERE EXISTS (
    <korreleret subquery>
);
```


In [ ]:
/* Nedenstående er eksempler på korrelerede queries der benytter EXISTS-prædikatet */

CREATE TABLE #TableA (
  Id int NOT NULL,
  TableB_Id int NULL
);

INSERT INTO #TableA (Id, TableB_Id)
VALUES
(1, 1), (2, 1), (3, 1),
(4, 2), (5, 2);

CREATE TABLE #TableB (
  Id int NULL,
  Col nvarchar(10) NULL
);

INSERT INTO #TableB (Id, Col)
VALUES
(1, 'A'), (2, 'B'), (3, 'B'),
(4, 'C');

SELECT * FROM #TableA;
SELECT * FROM #TableB;

/* I eksemplet findes transaktioner i #TableB med Col='B' og hvor der eksisterer (eller ikke
    eksisterer) tilhørende transaktioner i #TableA */

SELECT
    Id,
    Col
FROM #TableB as b
WHERE Col = 'B'
AND EXISTS (
--AND NOT EXISTS (
    SELECT *
    FROM #TableA AS a
    WHERE a.TableB_Id = b.Id
);

/* Mens brugen af asterisk, *, oftest er en dårlig praksis, så gælder dette ikke
for EXISTS - her bekymrer databasesystemet sig udelukkende om matchende rækker */

/* EXISTS performer godt! Databasesystemet ved godt at det udelukkende skal finde ud af om
der eksisterer mindst en matchende række eller ingen */

DROP TABLE #TableA;
DROP TABLE #TableB;

### *Tid til opgaver...*

Lav opgave 4, 5 og 6 i [opgavehæftet](Subqueries.sql).


## Hovedpointer

- Subqueries er en forespørgsel indlejret i en anden forespørgsel
- Subqueries kan have forskellige egenskaber. De kan være:
  - Uafhængige eller korrelede
  - Returnere en enkelt værdi eller en liste af værdier
- Subqueries bruges ofte fordi de er dynamiske og sparer nogle beregningstrin


## Licens

Creative Commons Attribution-ShareAlike 4.0 International (CC BY-SA 4.0)

Mere info: [https://creativecommons.org/licenses/by-sa/4.0/](https://creativecommons.org/licenses/by-sa/4.0/)

Du kan frit:

- Dele: kopiere og distribuere materialet via ethvert medium og i ethvert format
- Tilpasse: remixe, redigere og bygge på materialet til ethvert formål, selv erhvervsmæssigt

Under følgende betingelser:

- Kreditering: Du skal kreditere, dele et link til licensen og indikere om der er lavet ændringer.
- Del på samme vilkår: Hvis du remixer, redigerer eller bygger på materialet, så skal dine bidrag
  distribueres under samme licens som den originale.
